In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import pyplot as plt
from collections import defaultdict

class PlotHelper():
    def __init__(self):
        self.reset()

    def reset(self):
        self._f = None
        self._ax = None
        self.kvals = defaultdict(list)

    def add(self, **kval):
        for k, v in kval.items():
            self.kvals[k].append(v)

    @property
    def fig(self):
        if self._f is None:
            self.new()
        return self._f

    @property
    def ax(self):
        if self._ax is None:
            self.new()
        return self._ax

    def new(self):
        self._f, self._ax = plt.subplots(1,1)
        plt.ion()
        self.fig.show()

    def show(self):
        names = []
        self.ax.clear()
        for k, v in self.kvals.items():
            names.append(k)
            self.ax.plot(v)
        self.ax.legend(names)
        self.fig.canvas.draw()

In [2]:
def sample_balanced_subset(dataset, n_per_class):
    classes = dataset.targets.unique()
    indices = [(dataset.targets == c).nonzero(as_tuple=False) [:n_per_class] for c in classes]
    indices = torch.stack(indices).flatten()
    return torch.utils.data.Subset(dataset, indices)

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 100)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return x


def train(args, model, device, train_loader, optimizer, epoch, cons=None, plot_loss=None):
    
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        
        output = model(data)
        output = output.reshape(10,10)
        
        loss = F.cross_entropy(output[:1], target)
        closs = cons(output) if cons else torch.tensor(0).to(device)
        
        loss += closs
            
        loss.backward()
        optimizer.step()
        
        # Logging
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    
    test_loss = 0
    correct = 0
    with torch.no_grad():
        
        for data, target in test_loader:
            
            data, target = data.to(device), target.to(device)
            output = model(data)
            output = output.reshape(10,10)
            # sum up batch loss
            test_loss += F.cross_entropy(output[:1], target, reduction='sum').item()
            
            # get the index of the max log-probability
            pred = output[:1].argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [4]:
import sys
sys.path.append("..")

from pytorch_constraints.constraint import constraint
from pytorch_constraints.sampling_solver import WeightedSamplingSolver

def only_one(x):
    
    if not torch.is_tensor(x):
        x = torch.tensor(x)
        
    x = torch.zeros(10).scatter_(0, x, 1.)
    return sum(x) == 1

class Args:
    batch_size = 1
    test_batch_size = 1000
    epochs = 1
    lr = 1.0
    gamma = 0.7
    seed = 1
    log_interval = 10
    use_cuda = False
    n_per_class = 100
    
# Plotting
plot_loss = PlotHelper()
    
args = Args()
torch.manual_seed(args.seed)

device = torch.device("cuda" if args.use_cuda else "cpu")

kwargs = {'batch_size': args.batch_size}
if args.use_cuda:
    kwargs.update({'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True},
                 )

# Prepare dataset transformations
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

# Load train and test splits
train_split = datasets.MNIST('../data', train=True, download=True,
                   transform=transform)
test_split = datasets.MNIST('../data', train=False,
                   transform=transform)

# Sample a balanced subset of the train set
train_split = sample_balanced_subset(train_split, args.n_per_class)

# Create train, validation and test data loaders
train_loader = torch.utils.data.DataLoader(train_split,**kwargs)
test_loader  = torch.utils.data.DataLoader(test_split, **kwargs)

# Constraint to be applied on unlabeled data
only_one_constraint = constraint(only_one, WeightedSamplingSolver(num_samples=100))

# Move model to correct device
model = Net().to(device)

# Set up optimizers
optimizer = optim.Adadelta(model.parameters(), lr=args.lr)
scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)

In [5]:
for data, target in train_loader:
    break

for _ in range(500):
    
    optimizer.zero_grad()
    
    output = model(data)
    output = output.reshape(10,10)
    
    loss = F.cross_entropy(output[:1], target)
    closs = only_one_constraint(output)
    print(closs)
    
    loss += closs
#     print(loss)
    loss.backward()
    optimizer.step()

tensor(-0., grad_fn=<NegBackward>)
tensor(2.3842e-07, grad_fn=<NegBackward>)
tensor(0.0010, grad_fn=<NegBackward>)
tensor(0.0355, grad_fn=<NegBackward>)
tensor(8.3681e-05, grad_fn=<NegBackward>)
tensor(0.0516, grad_fn=<NegBackward>)
tensor(2.8133e-05, grad_fn=<NegBackward>)
tensor(0.1261, grad_fn=<NegBackward>)
tensor(0.0279, grad_fn=<NegBackward>)
tensor(1.0165, grad_fn=<NegBackward>)
tensor(0.0007, grad_fn=<NegBackward>)
tensor(0.0190, grad_fn=<NegBackward>)
tensor(1.3947e-05, grad_fn=<NegBackward>)
tensor(0.0158, grad_fn=<NegBackward>)
tensor(0.1750, grad_fn=<NegBackward>)
tensor(0.0002, grad_fn=<NegBackward>)
tensor(0.2606, grad_fn=<NegBackward>)
tensor(0.0010, grad_fn=<NegBackward>)
tensor(0.0046, grad_fn=<NegBackward>)
tensor(0.0009, grad_fn=<NegBackward>)
tensor(0.0024, grad_fn=<NegBackward>)
tensor(0.0031, grad_fn=<NegBackward>)
tensor(0.0027, grad_fn=<NegBackward>)
tensor(0.0011, grad_fn=<NegBackward>)
tensor(0.0144, grad_fn=<NegBackward>)
tensor(6.9973e-05, grad_fn=<NegBackwa

tensor(7.4741e-05, grad_fn=<NegBackward>)
tensor(0.0056, grad_fn=<NegBackward>)
tensor(3.7550e-05, grad_fn=<NegBackward>)
tensor(0.0002, grad_fn=<NegBackward>)
tensor(0.0002, grad_fn=<NegBackward>)
tensor(0.0002, grad_fn=<NegBackward>)
tensor(3.4928e-05, grad_fn=<NegBackward>)
tensor(4.5299e-06, grad_fn=<NegBackward>)
tensor(0.0001, grad_fn=<NegBackward>)
tensor(0.0002, grad_fn=<NegBackward>)
tensor(1.2278e-05, grad_fn=<NegBackward>)
tensor(0.0003, grad_fn=<NegBackward>)
tensor(0.0053, grad_fn=<NegBackward>)
tensor(0.0003, grad_fn=<NegBackward>)
tensor(0.0002, grad_fn=<NegBackward>)
tensor(8.4635e-05, grad_fn=<NegBackward>)
tensor(0.0005, grad_fn=<NegBackward>)
tensor(0.0001, grad_fn=<NegBackward>)
tensor(9.6912e-05, grad_fn=<NegBackward>)
tensor(0.0001, grad_fn=<NegBackward>)
tensor(0.0001, grad_fn=<NegBackward>)
tensor(0.0005, grad_fn=<NegBackward>)
tensor(0.0001, grad_fn=<NegBackward>)
tensor(1.6332e-05, grad_fn=<NegBackward>)
tensor(5.6027e-05, grad_fn=<NegBackward>)
tensor(9.4175e

tensor(2.5034e-06, grad_fn=<NegBackward>)
tensor(7.3549e-05, grad_fn=<NegBackward>)
tensor(5.9126e-05, grad_fn=<NegBackward>)
tensor(3.9100e-05, grad_fn=<NegBackward>)
tensor(2.0027e-05, grad_fn=<NegBackward>)
tensor(1.1563e-05, grad_fn=<NegBackward>)
tensor(0.0003, grad_fn=<NegBackward>)
tensor(3.9100e-05, grad_fn=<NegBackward>)
tensor(0.0001, grad_fn=<NegBackward>)
tensor(2.3961e-05, grad_fn=<NegBackward>)
tensor(1.7404e-05, grad_fn=<NegBackward>)
tensor(0.0001, grad_fn=<NegBackward>)
tensor(5.0186e-05, grad_fn=<NegBackward>)
tensor(2.0027e-05, grad_fn=<NegBackward>)
tensor(2.2173e-05, grad_fn=<NegBackward>)
tensor(2.3961e-05, grad_fn=<NegBackward>)
tensor(6.0797e-06, grad_fn=<NegBackward>)
tensor(4.2438e-05, grad_fn=<NegBackward>)
tensor(0.0003, grad_fn=<NegBackward>)
tensor(0.0002, grad_fn=<NegBackward>)
tensor(0.0002, grad_fn=<NegBackward>)
tensor(0.0001, grad_fn=<NegBackward>)
tensor(0.0058, grad_fn=<NegBackward>)
tensor(3.7193e-05, grad_fn=<NegBackward>)
tensor(4.2318e-05, grad_

In [6]:
torch.argmax(torch.softmax(output, dim=-1), dim=-1)

tensor([0, 4, 7, 7, 2, 7, 6, 6, 4, 3])

In [ ]:
# for epoch in range(1, args.epochs + 1):
#     train(args, model, device, train_loader, optimizer, epoch, only_one_constraint, plot_loss)
#     test(model, device, test_loader)
#     scheduler.step()

In [ ]:
# import sys
# sys.path.append("..")

# from pytorch_constraints.constraint import constraint
# from pytorch_constraints.sampling_solver import WeightedSamplingSolver

# def only_one(x):
    
#     if not torch.is_tensor(x):
#         x = torch.tensor(x)
        
#     x = torch.zeros(10).scatter_(0, x, 1.)
#     return sum(x) == 1

# class Args:
#     batch_size = 1
#     test_batch_size = 1000
#     epochs = 1
#     lr = 1.0
#     gamma = 0.7
#     seed = 1
#     log_interval = 10
#     use_cuda = False
#     n_per_class = 100
    
# # Plotting
# plot_loss = PlotHelper()
    
# args = Args()
# torch.manual_seed(args.seed)

# device = torch.device("cuda" if args.use_cuda else "cpu")

# kwargs = {'batch_size': args.batch_size}
# if args.use_cuda:
#     kwargs.update({'num_workers': 1,
#                    'pin_memory': True,
#                    'shuffle': True},
#                  )

# # Prepare dataset transformations
# transform=transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.1307,), (0.3081,))
#     ])

# # Load train and test splits
# train_split = datasets.MNIST('../data', train=True, download=True,
#                    transform=transform)
# test_split = datasets.MNIST('../data', train=False,
#                    transform=transform)

# # Sample a balanced subset of the train set
# train_split = sample_balanced_subset(train_split, args.n_per_class)

# # Create train, validation and test data loaders
# train_loader = torch.utils.data.DataLoader(train_split,**kwargs)
# test_loader  = torch.utils.data.DataLoader(test_split, **kwargs)

# # Constraint to be applied on unlabeled data
# only_one_constraint = constraint(only_one, WeightedSamplingSolver())

# # Move model to correct device
# model = Net().to(device)

# # Set up optimizers
# optimizer = optim.Adadelta(model.parameters(), lr=args.lr)
# scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)

# for epoch in range(1, args.epochs + 1):
#     train(args, model, device, train_loader, optimizer, epoch, only_one_constraint, plot_loss)
#     test(model, device, test_loader)
#     scheduler.step()